In [1]:
# importing the requried libraries

from collections import OrderedDict    # For recording the model specification 
import pandas as pd                    # For file input/output
import numpy as np                     # For vectorized math operations
import pylogit as pl                   # For MNL model estimation and conversion from wide to long format
import warnings
warnings.filterwarnings("ignore")

from  sympy import *

import math 

import matplotlib.pyplot as plt

In [2]:
# Ben Akiva
# Mathew

In [3]:
df = pd.read_csv('Northern_Khobar_made_up_data.csv')

In [4]:
# Create the list of individual specific variables
ind_variables = df.columns.tolist()[1:7]

# Specify the variables that vary across individuals and some or all alternatives
# The keys are the column names that will be used in the long format dataframe.
# The values are dictionaries whose key-value pairs are the alternative id and
# the column name of the corresponding column that encodes that variable for
# the given alternative. Examples below.


alt_varying_variables = { u'travel_cost': dict([(1, 'Cost-Car(SR)'),
                                                (2, 'Cost-PT(SR)')])  
                        }

# Specify the availability variables
# Note that the keys of the dictionary are the alternative id's.
# The values are the columns denoting the availability for the
# given mode in the dataset.
availability_variables = {1: 'CAR_AV',
                          2: 'PT_AV' 
                          }

##########
# Determine the columns for: alternative ids, the observation ids and the choice
##########
# The 'custom_alt_id' is the name of a column to be created in the long-format data
# It will identify the alternative associated with each row.
custom_alt_id = "mode_id"

# Create a custom id column that ignores the fact that this is a 
# panel/repeated-observations dataset. Note the +1 ensures the id's start at one.
obs_id_column = "custom_id"
df[obs_id_column] = np.arange(df.shape[0], dtype=int) + 1

# Create a variable recording the choice column
choice_column = "Choice"

In [5]:
# Perform the conversion to long-format
long_df = pl.convert_wide_to_long(df, 
                                           ind_variables, 
                                           alt_varying_variables, 
                                           availability_variables, 
                                           obs_id_column, 
                                           choice_column,
                                           new_alt_id_name=custom_alt_id)
# Look at the resulting long-format dataframe
long_df.head(10).T

,0,1,2,3,4,5,6,7,8,9
custom_id,1,1,2,2,3,3,4,4,5,5
mode_id,1,2,1,2,1,2,1,2,1,2
Choice,0,1,0,1,1,0,1,0,0,1
Cost-Car(SR),3,3,4,4,4,4,5,5,6,6
Cost-PT(SR),2,2,2,2,2,2,2,2,2,2
Time-Car(mins),6,6,5,5,10,10,9,9,8,8
Walk_Time-PT(mins),3,3,2,2,4,4,3,3,4,4
Wait_Time-PT(mins),3,3,5,5,4,4,4,4,2,2
Time_Ride-PT(mins),5,5,4,4,8,8,8,8,7,7
travel_cost,3,2,4,2,4,2,5,2,6,2


In [6]:
basic_specification = OrderedDict()
basic_names = OrderedDict()

# basic_specification["Summer(1=yes,0=no)"] = [1]

# basic_names["Summer(1=yes,0=no)"] = ['Summer Dummy = 1 if yes 0 if no'] 

basic_specification["Time-Car(mins)"] = [1]

basic_names["Time-Car(mins)"] = ['Travel Time Car (min)']  

basic_specification["Time_Ride-PT(mins)"] = [2]

basic_names["Time_Ride-PT(mins)"] = ['Travel Time Bus Trip (min)'] 


basic_specification["Walk_Time-PT(mins)"] = [2]

basic_names["Walk_Time-PT(mins)"] = ['Travel Time Walk to Bus Station (min)'] 

basic_specification["Wait_Time-PT(mins)"] = [2]

basic_names["Wait_Time-PT(mins)"] = ['Travel Time Wait at Bus Station (min)'] 


basic_specification["travel_cost"] = [1,2]

basic_names["travel_cost"] = [ 'Travel Cost, Car (SR)', 'Travel Cost, PT (SR)']    

basic_specification["travel_cost"] = [[1,2]]

basic_names["travel_cost"] = [ 'Travel Cost (SR)']    

In [7]:
khobarLogit = pl.create_choice_model(data=long_df, alt_id_col=custom_alt_id, obs_id_col=obs_id_column,
                                        choice_col=choice_column, specification=basic_specification,
                                        model_type="MNL", names=basic_names)

# Specify the initial values and method for the optimization.
khobarLogit.fit_mle(np.zeros(5)) # zeros(x) with x being the total number of parameters to be estimated

# Look at the estimation results
khobarLogit.get_statsmodels_summary()

Log-likelihood at zero: -20.7944
Initial Log-likelihood: -20.7944
Estimation Time for Point Estimation: 0.02 seconds.
Final log-likelihood: -16.1877


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      Choice   No. Observations:                   30
Model:             Multinomial Logit Model   Df Residuals:                       25
Method:                                MLE   Df Model:                            5
Date:                     Tue, 27 Oct 2020   Pseudo R-squ.:                   0.222
Time:                             18:35:10   Pseudo R-bar-squ.:              -0.019
AIC:                                42.375   Log-Likelihood:                -16.188
BIC:                                49.381   LL-Null:                       -20.794
=========================================================================================================
                                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Travel Time Car (min)                    -1.1970      0.840     -1.425      0.154      -2.844       0.450
Travel Time Bus Trip (min)               -1.2204      0.782     -1.560      0.119      -2.754       0.313
Travel Time Walk to Bus Station (min)    -0.7329      0.697     -1.052      0.293      -2.098       0.633
Travel Time Wait at Bus Station (min)    -0.1165      0.235     -0.495      0.621      -0.578       0.345
Travel Cost (SR)                         -0.6234      0.315     -1.980      0.048      -1.241      -0.006
=========================================================================================================
"""

In [8]:
VOT_Car  = khobarLogit.params["Travel Time Car (min)"]/khobarLogit.params["Travel Cost (SR)"]
VOT_Bus  = khobarLogit.params["Travel Time Bus Trip (min)"]/khobarLogit.params["Travel Cost (SR)"]
VOT_Walk = khobarLogit.params["Travel Time Walk to Bus Station (min)"]/khobarLogit.params["Travel Cost (SR)"]
VOT_Wait = khobarLogit.params["Travel Time Wait at Bus Station (min)"]/khobarLogit.params["Travel Cost (SR)"]

In [9]:
print("Value of time spent in a car               =", VOT_Car.round(2), "SR/min")
print("Value of time spent in a bus               =", VOT_Bus.round(2), "SR/min")
print("Value of time spent walking to bus station =", VOT_Walk.round(2),"SR/min")
print("Value of time spent waiting at bus station =", VOT_Wait.round(2),"SR/min")

Value of time spent in a car               = 1.92 SR/min
Value of time spent in a bus               = 1.96 SR/min
Value of time spent walking to bus station = 1.18 SR/min
Value of time spent waiting at bus station = 0.19 SR/min


In [10]:
print("Value of time spent in a car               =", (VOT_Car*60).round(1), "SR/hr")
print("Value of time spent in a bus               =", (VOT_Bus*60).round(1), "SR/hr")
print("Value of time spent walking                =", (VOT_Walk*60).round(1),"SR/hr")
print("Value of time spent waiting at bus station =", (VOT_Wait*60).round(2),"SR/hr")

Value of time spent in a car               = 115.2 SR/hr
Value of time spent in a bus               = 117.5 SR/hr
Value of time spent walking                = 70.5 SR/hr
Value of time spent waiting at bus station = 11.21 SR/hr


In [11]:
VOT_Car  =  VOT_Car  * 60
VOT_Bus  =  VOT_Bus  * 60
VOT_Walk =  VOT_Walk * 60 
VOT_Wait =  VOT_Wait * 60

## Using Value of Time in Deganzo's equations. 

Introduce daganzo equations
Latec.

In [12]:
cd, ct, cm, lambda_ ,S ,s, H, B, k , delta, vw, vmax, l,ts   = symbols('c_d c_t c_m lambda S s H beta k Delta v_w v_max l t_s')

In [13]:
# Objective funciton as specified in Deganzo (20xx) -- Predicated on adding supply and demand costs to a sole function.
objective_function = 4*cd/(lambda_*S*H) + B*(k*H/2 + delta + (S+s)/(2*vw) + l/vmax + (l*ts)/s)

In [14]:
objective_function

beta*(Delta + H*k/2 + l/v_max + l*t_s/s + (S + s)/(2*v_w)) + 4*c_d/(H*S*lambda)

In [15]:
# Beta is the value of time (VOT) given by SR/min. I'll be using VOT for bus ride VOT, walking VOT and waiting VOT to 
# change trip duration and trip distance to units of cost.

ct = 40             # cost of bus while stopped -- Operator salary [SR/hr]

cm = 12             # cost of maintenance + operation + depreciation [SR/ vehicle-km] -> 
                    # -> This can be approximated by th sum of vehicle cost to operate + mainetnance cost - salvage value
                    # and the average kilometers driven before being decommissioned.
                    # http://www.freightmetrics.com.au/Calculators%7CRoad/BusOperatingCost/tabid/671/Default.aspx
            
vmax = 40           # Bus max speed [km/hr].       
vw = 4              # Walking speed [km/hr].
cd = cm + ct/vmax   # Cost of operating a guideway [SR/ vehicle-km].
S  = S              # Spacing between transfer stops [km].
s  = s              # Spacing between stops [km].
H  = H              # Headway [minutes]
lambda_ = 10000     # Rate of trip generation given [trip/(km^2-hr)]
k  = 1              # k =1 if passengers know the schedule. k=2 if passengers don't. 
delta = 1/12        # Penalty for line transfers assumed as 5 mins.
L,W = 1.5, 2.5      # Dimensions of Northern khobar neighborhood as calculted by Qgis [km]
l =   L/3 + W/3     # For a trip starting and ending at two random points on a corridor of length l, the average length
                    # of one trip is given by l/3. Since we expect the average trip to comprise a
                    # a single vertical and a single horizontal movement along the network.  
                    # the average length of each trip is given as: L/3 + W/3.
                    # math.stackexchange.com/questions/195245/average-distance-between-random-points-on-a-line-segment
ts =    0.005       # Average Stop time. Time to alight and board [hr] -> Just an assumption.

In [16]:
#objective_function = Supply side cost + Demand side cost.
objective_function = 4*cd/(lambda_*S*H) + (VOT_Wait*(k*H/2 + delta) + VOT_Walk*(S+s)/(2*vw) + VOT_Bus *(l/vmax + (l*ts)/s))

## Now that we've constructed our objective function, we'd like to find the set of variables[S,s,H] that minimize our objective function. 

In [18]:
def cost(listz, obj): 
    
    S = listz[0]
    s = listz[1]
    H = listz[2]
    
    return eval(obj)

In [19]:
def const1(listz):
    
    return listz[0] - listz[1] 

In [20]:
x0 = [1,1,20]
cost(x0,str(objective_function))

135.40431989527133

In [27]:
import scipy

bnds = [(0, 10), (0, 10), (0, 1/6)]
cons = [{'type': 'ineq', 'cost': const1}]

scipy.optimize.minimize(cost, [0.2,0.2,0.1], args = (str(objective_function)), bounds = bnds)#, constraints = cons)

      fun: 13.112402233523886
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([  7.51719735, -10.75871108,   3.00684899])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 12
      nit: 1
   status: 0
  success: True
        x: array([0.2, 0.2, 0.1])

In [28]:
S_test = np.arange(0.05,1,0.05)
s_test = np.arange(0.05,1,0.05)
H_test = np.arange(0.001,11/60,1/60)
result = []

In [29]:
for S in S_test:
    for s in s_test:
        for H in H_test:
            result.append([S,s,H,cost([S,s,H],str(objective_function))])

In [30]:
result= np.array(result)

In [31]:
result[np.argmin(result[:,3]),:]

array([ 0.1       ,  0.3       ,  0.101     , 12.06779994])

In [32]:
S = 0.5
s = 0.25
H = 10/60

cost([S,s,H],str(objective_function))

15.59157474526589